In [10]:
import re
import os
import yaml
import json
import pandas as pd
from google.cloud import bigquery
import google.api_core.exceptions as google_exceptions

In [11]:
config  = yaml.safe_load(open('config.yaml', 'r'))

In [12]:
query_dir = 'queries'

In [13]:
queries = {}
for query_file in (os.listdir(query_dir)):
    with open(os.path.join(query_dir, query_file), 'r') as query:
        queries[query_file] = query.read()

In [14]:
query_client = bigquery.Client()

In [15]:
def exists_table(table_reference, client):
    try:
        client.get_table(table_reference)
        return True
    except google_exceptions.NotFound:
        return False

In [16]:
if exists_table(f'{config["project_id"]}.{config["dataset"]}.expanded_sub_properties', query_client):
    request = queries['all_new_buildings_that_have_not_been_expanded.sql']
else:
    request = queries['all_new_buildings.sql']

In [17]:
real_estate_raw = query_client.query(request).to_dataframe()

In [18]:
real_estate_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 5 columns):
ad_id             45 non-null int64
new_building      45 non-null bool
property_table    45 non-null object
property_attrs    45 non-null object
address           45 non-null object
dtypes: bool(1), int64(1), object(3)
memory usage: 1.5+ KB


In [19]:
real_estate_raw.set_index(keys='ad_id', drop=True, inplace=True)

In [20]:
real_estate_raw.head()

,new_building,property_table,property_attrs,address
ad_id,,,,
77962792,True,"{""Bolig"": [""21"", ""27"", ""28"", ""29"", ""30"", ""32"",...","{""Omkostninger"": ""22\u00a0665 kr"", ""Totalpris""...","Gamle Lommedalsvei 123, 1348 Rykkinn"
96760455,True,"{""Bolig"": [""B401"", ""C201"", ""C202"", ""C301"", ""C3...","{""Omkostninger"": ""1\u00a0064 kr"", ""Totalpris"":...","Røa Torg - Vækerøveien 195, 0751 Oslo"
96760455,True,"{""Bolig"": [""A303"", ""B401"", ""C201"", ""C202"", ""C3...","{""Omkostninger"": ""1\u00a0064 kr"", ""Totalpris"":...","Røa Torg - Vækerøveien 195, 0751 Oslo"
96760455,True,"{""Bolig"": [""A303"", ""B401"", ""C201"", ""C202"", ""C3...","{""Omkostninger"": ""1\u00a0064 kr"", ""Totalpris"":...","Røa Torg - Vækerøveien 195, 0751 Oslo"
96760455,True,"{""Bolig"": [""A303"", ""B401"", ""C201"", ""C202"", ""C3...","{""Omkostninger"": ""1\u00a0064 kr"", ""Totalpris"":...","Røa Torg - Vækerøveien 195, 0751 Oslo"


In [21]:
for col in ['property_table', 'property_attrs']:
    real_estate_raw[col] = [json.loads(cell) for cell in real_estate_raw[col]]

In [22]:
def expanand_property_tables(new_buildings):
    cols = new_buildings.iloc[0]['property_table'].keys()
    col_map = {'Bolig': 'apt_id',
               'Soverom': 'num_bedrooms',
               'Etg': 'floor',
               'P-rom': 'primary_size',
               'BRA': 'total_size',
               'Totalpris': 'price',
               'Pris': 'price'}
    property_tables = pd.DataFrame(columns=cols)
    for code, row in new_buildings['property_table'].iteritems():
        all_properties = [arr for arr in row.values()]
        to_append = dict(zip(cols, all_properties))
        frame = pd.DataFrame(to_append, columns=cols)
        frame['ad_id'] = code
        property_tables = property_tables.append(frame, ignore_index=True, sort=False)
    property_tables.rename(columns=col_map, inplace=True)
    return property_tables

In [23]:
sub_properties = expanand_property_tables(real_estate_raw[(real_estate_raw.new_building == True) & (real_estate_raw.property_table)])

In [24]:
sub_properties.head()

,apt_id,num_bedrooms,floor,primary_size,total_size,price,ad_id
0,21,2,1.,66 m²,71 m²,4 418 000 kr,77962792.0
1,27,2,1.,67 m²,71 m²,4 518 200 kr,77962792.0
2,28,2,1.,67 m²,71 m²,4 551 700 kr,77962792.0
3,29,2,1.,66 m²,71 m²,4 554 400 kr,77962792.0
4,30,2,1.,66 m²,71 m²,4 569 400 kr,77962792.0


In [25]:
def get_int_from_str(string):
    try:
        concat_string = ''.join(re.findall(r'[\d.]', string))
        if concat_string != '':
            return float(concat_string)
    except Exception as e:
        print(e, string)
        pass

In [26]:
sub_properties_parsed = sub_properties.copy()
for col in ['primary_size', 'floor', 'price', 'num_bedrooms', 'total_size']:
    sub_properties_parsed[col] = [get_int_from_str(s) for s in sub_properties_parsed[col]]

In [27]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571 entries, 0 to 570
Data columns (total 7 columns):
apt_id          571 non-null object
num_bedrooms    570 non-null float64
floor           534 non-null float64
primary_size    571 non-null float64
total_size      571 non-null float64
price           571 non-null float64
ad_id           571 non-null float64
dtypes: float64(6), object(1)
memory usage: 31.3+ KB


In [28]:
sub_properties_parsed.head()

,apt_id,num_bedrooms,floor,primary_size,total_size,price,ad_id
0,21,2.0,1.0,66.0,71.0,4418000.0,77962792.0
1,27,2.0,1.0,67.0,71.0,4518200.0,77962792.0
2,28,2.0,1.0,67.0,71.0,4551700.0,77962792.0
3,29,2.0,1.0,66.0,71.0,4554400.0,77962792.0
4,30,2.0,1.0,66.0,71.0,4569400.0,77962792.0


In [29]:
sub_properties_parsed.drop_duplicates(inplace=True)

In [30]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313 entries, 0 to 570
Data columns (total 7 columns):
apt_id          313 non-null object
num_bedrooms    312 non-null float64
floor           289 non-null float64
primary_size    313 non-null float64
total_size      313 non-null float64
price           313 non-null float64
ad_id           313 non-null float64
dtypes: float64(6), object(1)
memory usage: 19.6+ KB


In [31]:
sub_properties_parsed.to_gbq(destination_table='housing_data.expanded_sub_properties',
                             project_id='hde-test-clean',
                             if_exists='append')